In [1]:
from src.recognizer import Recognizer
import numpy as np
import cv2
import os
import glob

In [2]:
# Set your own paths
cross_directory = r'C:\Users\Vlad\Desktop\Го\diagram-recognition\dataset\cross'
white_directory = r'C:\Users\Vlad\Desktop\Го\diagram-recognition\dataset\white'
black_directory = r'C:\Users\Vlad\Desktop\Го\diagram-recognition\dataset\black'

dataset_size = 500
cell_size_multiplier = 0.6
paths = ['data/images/pages/chinese/1000_TsumeGo-6.png']
save_crosses = True # Setting to true will slow calculations a bit even if we have dataset_size crosses

In [3]:
def set_diff2d(A, B):
    nrows, ncols = A.shape
    dtype={'names':['f{}'.format(i) for i in range(ncols)], 'formats':ncols * [A.dtype]}
    C = np.setdiff1d(A.copy().view(dtype), B.copy().view(dtype))
    return C

def process_items(board, items, delta):
    result = []
    for item in items:
        result.append(process_item(board, item, delta))
    return result

def process_item(board, item, delta):
    y = item[0]
    x = item[1]
    item_img = board[max(x-delta,0):min(x+delta,board.shape[0]),max(y-delta,0):min(y+delta,board.shape[1])]
    return item_img

def save_items(directory, items, force_save = False):
    latest_file_index = len(os.listdir(directory))
    if (not force_save) and (latest_file_index > dataset_size):
        return
    os.chdir(directory)
    quantity_to_save = min(len(items), dataset_size - latest_file_index)
    for i in range(quantity_to_save):
        filename = str(latest_file_index+i) + ".png"
        cv2.imwrite(filename, items[i])
    
def process_boards(boards, save_crosses):
    for board in boards:
        img_gray = cv2.cvtColor(board, cv2.COLOR_BGR2GRAY)
        v_lines, h_lines = rec.lines_recognition(img_gray)
        x_size, y_size = v_lines.shape[0], h_lines.shape[0]
        intersections = rec.find_intersections(v_lines, h_lines)
        cell_size = rec.get_cell_size(v_lines, h_lines)

        stones, radius = rec.stones_recognition(img_gray, cell_size, intersections)
        white_stones, black_stones = rec.colorize(img_gray, stones, radius)
        intersections = np.reshape(intersections, (-1, 2))
        delta = int(cell_size * cell_size_multiplier)

        if save_crosses:
            crosses = set_diff2d(intersections, np.concatenate((white_stones, black_stones)))
            cross_imgs = process_items(board, crosses, delta)
            save_items(cross_directory, cross_imgs)

        white_imgs = process_items(board, white_stones, delta)
        save_items(white_directory, white_imgs)

        black_imgs = process_items(board, black_stones, delta)
        save_items(black_directory, black_imgs)

In [4]:
rec = Recognizer()

for path in paths:
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    boards = rec.split_into_boards(img)
    process_boards(boards, save_crosses)